In [3]:
# imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [4]:
df = pd.read_csv('C:\\Users\\willm\\Desktop\\college_biz\\Data\\final_school_data.csv')

In [5]:
# Target variable
target = "happiness"

In [6]:
# drop high-cardinality identifiers
drop_cols = [
    "rmp_school_id",
    "city",
    "overall_rating",
    "state",
    "number_of_ratings",
    "reputation",
    "campus_setting",
    "sat_median_total",
    "act_median_composite",
    "acceptance_rate",
    "avg_aid_awarded",
    "total_expenses_in_state",
    "total_expenses_out_state",
    "student_population_total",
    "student_to_faculty_ratio",
    "retention_rate_avg",
    "grad_rate_4yr",
]
df = df.drop(columns=drop_cols, errors="ignore")

In [7]:
# scale happiness to [0,1] using the true theoretical range 1 → 5
df[target] = (df[target] - 1.0) / 4.0

In [8]:
# features / target split
X = df.drop(columns=[target])   # target not used as input
y = df[target]

In [9]:
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()

In [10]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", MinMaxScaler(), numeric_cols),
    ]
)

In [11]:
# define different models to test
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(
        n_estimators=500,
        max_depth=10,
        min_samples_leaf=4,
        random_state=42,
        n_jobs=-1,
    ),
    "XGBoost": XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        n_jobs=-1,
        random_state=42,
    ),
}

In [12]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [13]:
# test model metrics
fitted_pipes = {}
results = []

for name, model in models.items():
    pipe = Pipeline([
        ("preprocess", preprocessor),
        ("model", model),
    ])

    pipe.fit(X_train, y_train)
    fitted_pipes[name] = pipe

    preds = pipe.predict(X_test)

    train_preds = pipe.predict(X_train)
    train_r2 = r2_score(y_train, train_preds)

    r2 = r2_score(y_test, preds)
    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))

    results.append([name, train_r2, r2, mae, rmse])

In [14]:
# print model results
print("\n\n===== MODEL PERFORMANCE LEADERBOARD =====")
results_df = pd.DataFrame(results, columns=["Model", "Train R2", "Test R2", "MAE", "RMSE"])
print(results_df.sort_values(by="Test R2", ascending=False))



===== MODEL PERFORMANCE LEADERBOARD =====
               Model  Train R2   Test R2       MAE      RMSE
1      Random Forest  0.871418  0.749085  0.063074  0.094205
0  Linear Regression  0.724025  0.738923  0.065121  0.096094
2            XGBoost  0.955992  0.718094  0.065727  0.099853


In [15]:
def get_feature_names_from_pipeline(pipe):
    pre = pipe.named_steps["preprocess"]
    return pre.get_feature_names_out()

print("\n\n===== FEATURE IMPORTANCE =====")

for name, pipe in fitted_pipes.items():
    print(f"\n==== {name} ====")

    feature_names = get_feature_names_from_pipeline(pipe)

    if name == "Linear Regression":
        importances = pipe.named_steps["model"].coef_
    elif name in ["Random Forest", "XGBoost"]:
        importances = pipe.named_steps["model"].feature_importances_
    else:
        continue

    fi = pd.DataFrame({
        "feature": feature_names,
        "importance": importances
    }).sort_values(by="importance", ascending=False)

    print(fi.head(25))



===== FEATURE IMPORTANCE =====

==== Linear Regression ====
              feature  importance
2  num__opportunities    0.245053
0     num__facilities    0.224881
4         num__social    0.212581
5         num__safety    0.175061
7       num__internet    0.097382
1       num__location    0.090695
6           num__food    0.045605
3          num__clubs    0.015460

==== Random Forest ====
              feature  importance
2  num__opportunities    0.370834
0     num__facilities    0.349139
4         num__social    0.089035
5         num__safety    0.073952
1       num__location    0.039939
7       num__internet    0.029545
3          num__clubs    0.025336
6           num__food    0.022220

==== XGBoost ====
              feature  importance
2  num__opportunities    0.296766
0     num__facilities    0.266304
4         num__social    0.130409
5         num__safety    0.095903
7       num__internet    0.056355
1       num__location    0.055917
3          num__clubs    0.050400
6         

In [16]:
# model chosen for what if analysis
rf_pipe = fitted_pipes["Random Forest"]

In [17]:
# cross validate rf model to check for overfitting

cv_scores = cross_val_score(rf_pipe, X, y, cv=5, scoring='r2', n_jobs=-1)
print("===== RANDOM FOREST CROSS-VALIDATION SCORES =====")
print(f"CV R2 scores: {cv_scores}")
print(f"Mean CV R2: {np.mean(cv_scores)}")
print(f"Std CV R2: {np.std(cv_scores)}")  

===== RANDOM FOREST CROSS-VALIDATION SCORES =====
CV R2 scores: [0.8082955  0.71046142 0.76193878 0.70694653 0.73130003]
Mean CV R2: 0.7437884513082776
Std CV R2: 0.037721539262990614


**Conclusion:** The Random Forest model shows strong and stable generalization (CV R² ≈ 0.74, Test R² ≈ 0.75) with only moderate, expected overfitting, making it a reliable and well-balanced model for what-if feature analysis.


In [18]:
# all numeric features considered controllable for now
numeric_controllable = [
    "facilities",
    "location",
    "opportunities",
    "clubs",
    "social",
    "safety",
    "food",
    "internet",
]

In [19]:
# helper function to get base feature row for a specific school
def get_school_base_row(school_name: str):
    """
    Return (row_full, base_row_df) for a given school.
    base_row_df is a 1-row DataFrame containing only model input features.
    """
    mask = (df["school_name"] == school_name)
    if not mask.any():
        raise ValueError(f"School '{school_name}' not found in df['school_name'].")

    row_full = df.loc[mask].iloc[0]
    # drop target and identifier to get pure model features
    row_X = row_full.drop(labels=[target, "school_name"])
    base_row_df = row_X.to_frame().T
    return row_full, base_row_df

In [20]:
# helper function to get the numeric MinMaxScaler from the pipeline
def get_numeric_scaler(pipe):
    """
    Extract the MinMaxScaler used for numeric columns from a fitted pipeline.
    """
    pre = pipe.named_steps["preprocess"]
    return pre.named_transformers_["num"]

In [21]:
def improvement_ranking_for_school(
    school_name,
    delta_change=0.1,   # 0.1 in scaled feature units = 10% of that feature's range
    top_n=9,
):
    """
    For one school, estimate how much *scaled* happiness (0–1) would increase
    if each controllable numeric feature were improved by `delta_change` in
    MinMax-scaled feature space, then rank features by gain.

    Since the model is trained on 0–1 happiness, all predictions and gains
    are already in scaled units.
    """

    # get the school's base feature row
    _, base_row_df = get_school_base_row(school_name)

    # base prediction (already in [0,1])
    base_pred_scaled = rf_pipe.predict(base_row_df)[0]

    # get scaled feature values from the pipeline's MinMaxScaler
    scaler = get_numeric_scaler(rf_pipe)

    row_num = base_row_df[numeric_cols]
    row_scaled = scaler.transform(row_num)    

    rows = []

    # loop over each controllable numeric feature
    for f in numeric_controllable:
        idx = numeric_cols.index(f)

        current_feat_scaled = float(row_scaled[0, idx])
        headroom = 1.0 - current_feat_scaled
        applied_change = min(delta_change, headroom)

        if applied_change <= 0:
            gain_scaled = 0.0
            projected_scaled = base_pred_scaled
        else:
            # bump this one feature in scaled space
            new_scaled = row_scaled.copy()
            new_scaled[0, idx] = current_feat_scaled + applied_change

            # inverse-transform to original feature units
            new_num = scaler.inverse_transform(new_scaled)

            # build changed feature row
            changed_row_df = base_row_df.copy()
            changed_row_df[numeric_cols] = new_num

            # new prediction in scaled happiness (0–1)
            after_scaled = rf_pipe.predict(changed_row_df)[0]
            gain_scaled = after_scaled - base_pred_scaled
            projected_scaled = after_scaled

        rows.append({
            "feature": f,
            "current_feature_scaled": current_feat_scaled,
            "feature_headroom": headroom,
            "applied_feature_change_scaled": applied_change,
            "gain_scaled": gain_scaled,                      # delta happiness (0–1)
            "gain_percent_points": gain_scaled * 100.0,      # percentage points
            "projected_happiness_scaled": projected_scaled,
            "projected_happiness_percent": projected_scaled * 100.0,
        })

    # build and sort table
    imp_df = pd.DataFrame(rows)
    imp_df = imp_df.sort_values("gain_scaled", ascending=False).reset_index(drop=True)

    print(f"School: {school_name}")
    print(f"Predicted current happiness: {base_pred_scaled*100:.2f}%")
    print(f"Impacts with applied change of {delta_change*100:.0f}% in feature scaled space:\n")

    display_cols = [
        "feature",
        "current_feature_scaled",
        "feature_headroom",
        "applied_feature_change_scaled",
        "gain_scaled",
        "gain_percent_points",
        "projected_happiness_scaled",
        "projected_happiness_percent",
    ]

    return imp_df[display_cols].head(top_n).round(4)

In [22]:
test_school = "Florida Polytechnic University"
improvement_ranking_for_school(
    test_school,
    delta_change=0.2,
    top_n=8,
)

School: Florida Polytechnic University
Predicted current happiness: 65.62%
Impacts with applied change of 20% in feature scaled space:



,feature,current_feature_scaled,feature_headroom,applied_feature_change_scaled,gain_scaled,gain_percent_points,projected_happiness_scaled,projected_happiness_percent
0,social,0.400,0.600,0.200,0.0544,5.4422,0.7106,71.0596
1,facilities,0.675,0.325,0.200,0.0544,5.4361,0.7105,71.0535
2,opportunities,0.650,0.350,0.200,0.0265,2.6520,0.6827,68.2694
3,food,0.375,0.625,0.200,0.0200,1.9966,0.6761,67.6140
4,safety,0.925,0.075,0.075,0.0196,1.9641,0.6758,67.5815
5,internet,0.600,0.400,0.200,0.0102,1.0169,0.6663,66.6343
6,location,0.425,0.575,0.200,0.0072,0.7173,0.6633,66.3347
7,clubs,0.525,0.475,0.200,0.0008,0.0794,0.6570,65.6968


In [23]:
# sweep of investment levels in scaled feature space (0–1)
DELTA_SWEEP = np.linspace(0.0, 0.5, 11)

In [24]:
def create_batch_perturbations(base_row_df, numeric_features, numeric_controllable, delta_sweep, pipe):
    """
    Given a single school's feature row (base_row_df), create a batch DataFrame
    where each row corresponds to:
        - one controllable feature
        - one delta in delta_sweep
        - that feature bumped in scaled (0–1) feature space

    Returns a DataFrame that includes:
        - all model input features
        - '__feature__', '__delta__', '__applied_change__' metadata columns
    """
    # get the scaler from the pipeline
    scaler = get_numeric_scaler(pipe)

    # numeric subset of the row in correct order
    row_num = base_row_df[numeric_features]
    row_scaled = scaler.transform(row_num)    

    batch_rows = []

    for f in numeric_controllable:
        idx = numeric_features.index(f)
        current_scaled = float(row_scaled[0, idx])
        headroom = 1.0 - current_scaled

        for delta in delta_sweep:
            applied_change = min(delta, headroom)

            # copy the scaled row and bump this feature
            new_scaled = row_scaled.copy()
            new_scaled[0, idx] = current_scaled + applied_change

            # back to original numeric units
            new_num = scaler.inverse_transform(new_scaled)

            # build full changed row 
            changed_row_df = base_row_df.copy()
            changed_row_df[numeric_features] = new_num

            
            changed_row_df["__feature__"] = f
            changed_row_df["__delta__"] = float(delta)
            changed_row_df["__applied_change__"] = float(applied_change)

            
            batch_rows.append(changed_row_df)

    batch_df = pd.concat(batch_rows, ignore_index=True)
    return batch_df

In [25]:
def get_max_happiness_per_delta(school_name, delta_sweep=None):
    """
    For a given school, determines which feature yields the MAX SCALED HAPPINESS
    at each delta step (investment level), using a single batch prediction call
    for efficiency.

    Model predictions are already in [0,1] happiness space.
    """

    if delta_sweep is None:
        delta_sweep = DELTA_SWEEP

    # locate the school row based on feature
    _, base_row_df = get_school_base_row(school_name)

    # build all perturbed rows in one batch
    batch_df_input = create_batch_perturbations(
        base_row_df=base_row_df,
        numeric_features=numeric_cols,
        numeric_controllable=numeric_controllable,
        delta_sweep=delta_sweep,
        pipe=rf_pipe,
    )

    # columns used by the model 
    feature_cols = [c for c in batch_df_input.columns if not c.startswith("__")]

    # single batch prediction
    batch_predictions_scaled = rf_pipe.predict(batch_df_input[feature_cols])

    # attach predictions
    results_df = batch_df_input[["__feature__", "__delta__", "__applied_change__"]].copy()
    results_df["projected_happiness_scaled"] = batch_predictions_scaled

    # filter out zero-change rows 
    results_df = results_df[results_df["__applied_change__"] > 1e-6]

    # for each delta pick feature with max projected happiness
    idx_max = results_df.groupby("__delta__")["projected_happiness_scaled"].idxmax()
    dominant = results_df.loc[idx_max].reset_index(drop=True)

    # final formatting
    dominant = dominant.rename(
        columns={"__delta__": "Delta_Investment", "__feature__": "Dominant_Feature"}
    )

    dominant["Projected_Happiness"] = dominant["projected_happiness_scaled"].apply(
        lambda x: f"{x*100:.3f}%"   # scaled to 0–100%
    )
    dominant["Delta_Investment"] = dominant["Delta_Investment"].apply(
        lambda x: f"{x:.3f}"
    )

    final_table = dominant[["Delta_Investment", "Dominant_Feature", "Projected_Happiness"]]

    print(f"--- Max Impact Feature Analysis for {school_name} ---")
    return final_table


In [26]:
get_max_happiness_per_delta(test_school)


--- Max Impact Feature Analysis for Florida Polytechnic University ---


,Delta_Investment,Dominant_Feature,Projected_Happiness
0,0.050,facilities,67.943%
1,0.100,facilities,70.412%
2,0.150,facilities,70.564%
3,0.200,social,71.060%
4,0.250,social,71.825%
5,0.300,social,72.302%
6,0.350,social,72.022%
7,0.400,social,73.111%
8,0.450,social,74.552%
9,0.500,social,75.519%


In [27]:
def get_marginal_gain_breakpoints(school_name, delta_sweep=None):
    """
    For a given school, analyzes the marginal gain (biggest jump in SCALED happiness)
    between consecutive delta steps for each feature, using a single batch prediction.

    Since the model is trained on 0–1 happiness, gains are directly in "fraction of
    full happiness range" units (later converted to percentage points).
    """

    if delta_sweep is None:
        delta_sweep = DELTA_SWEEP

    # locate the school and row based on features
    _, base_row_df = get_school_base_row(school_name)

    # base prediction (0–1)
    base_pred_scaled = rf_pipe.predict(base_row_df)[0]

    # build all perturbed rows in one batch
    batch_df_input = create_batch_perturbations(
        base_row_df=base_row_df,
        numeric_features=numeric_cols,
        numeric_controllable=numeric_controllable,
        delta_sweep=delta_sweep,
        pipe=rf_pipe,
    )

    # columns actually used by the model 
    feature_cols = [c for c in batch_df_input.columns if not c.startswith("__")]

    # batch prediction (0–1)
    batch_predictions_scaled = rf_pipe.predict(batch_df_input[feature_cols])

    results_df = batch_df_input[["__feature__", "__delta__"]].copy()
    results_df["projected_happiness_scaled"] = batch_predictions_scaled

    # marginal gain per feature between neighboring deltas
    marginal_df = results_df.sort_values(
        by=["__feature__", "__delta__"]
    ).reset_index(drop=True)

    # Previous scaled happiness per feature (first step uses base_pred_scaled)
    marginal_df["prev_scaled"] = (
        marginal_df
        .groupby("__feature__")["projected_happiness_scaled"]
        .shift(1)
        .fillna(base_pred_scaled)
    )

    marginal_df["marginal_gain"] = (
        marginal_df["projected_happiness_scaled"] - marginal_df["prev_scaled"]
    )

    # find the delta with max marginal gain for each feature
    idx_max = marginal_df.groupby("__feature__")["marginal_gain"].idxmax()
    marginal_breakpoints = marginal_df.loc[idx_max].reset_index(drop=True)


    marginal_breakpoints = marginal_breakpoints.rename(
        columns={
            "__delta__": "Optimal_Delta",
            "marginal_gain": "Max_Marginal_Gain",
            "__feature__": "feature",
        }
    )

    # Convert marginal gain to percentage points (string) and delta to string
    marginal_breakpoints["Max_Marginal_Gain"] = (
        marginal_breakpoints["Max_Marginal_Gain"] * 100.0
    ).apply(lambda x: f"{x:.3f}%")

    marginal_breakpoints["Optimal_Delta"] = marginal_breakpoints["Optimal_Delta"].apply(
        lambda x: f"{x:.3f}"
    )

    # Drop features with zero marginal gain
    final_table = marginal_breakpoints[
        marginal_breakpoints["Max_Marginal_Gain"] != "0.000%"
    ].sort_values("Max_Marginal_Gain", ascending=False).reset_index(drop=True)

    print(f"\n--- Marginal Gain Breakpoint Analysis for {school_name} ---")
    return final_table

In [28]:
get_marginal_gain_breakpoints(test_school)



--- Marginal Gain Breakpoint Analysis for Florida Polytechnic University ---


,feature,Optimal_Delta,projected_happiness_scaled,prev_scaled,Max_Marginal_Gain
0,facilities,0.100,0.704118,0.679425,2.469%
1,opportunities,0.100,0.678701,0.656898,2.180%
2,social,0.200,0.710596,0.690785,1.981%
3,location,0.250,0.682775,0.663347,1.943%
4,safety,0.050,0.673266,0.656174,1.709%
5,food,0.100,0.669637,0.662777,0.686%
6,internet,0.050,0.661206,0.656174,0.503%
7,clubs,0.100,0.656693,0.655595,0.110%


In [29]:
# THINGS TO ADD:
# function - change to this item usual comes with a X% increase in other item, show how much overall happiness goes up when both are changed
# clean up comments 
# visualizations in AWS